In [1]:
import pandas as pd

## My API key  & library_Id are stored in another file
api_key,library_id = pd.read_json("Zotero_API_key.json", typ='series')

library_type="group"


def print_time():
    from datetime import datetime
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("Current Time =", current_time)


from pyzotero import zotero
zot = zotero.Zotero(library_id, library_type, api_key)

ID_Universities='EWSYI3RS' # to access the files in the university subfolder.

In [ ]:
for i in zot.collections_sub(ID_Universities):
    name,key=i["data"]["name"],i["data"]["key"]
    print(f"name: {name} --> key: {key}")

In [ ]:
def relevantItemInfo(dictionary):    
    if dictionary["itemType"]=='attachment':
        # if directory --> key leads to the correct folder
        
        relevant_keys=['key','url']
        newDict={key: dictionary[key] for key in relevant_keys}
        #print(dictionary)
        newDict["note"]=None
       # print(newDict)
        return newDict
    
    if dictionary["itemType"]=="note":
        relevant_keys=['key','note']
        newDict={key: dictionary[key] for key in relevant_keys}
        newDict["url"]=None
        return newDict

    

for i in zot.collection_items(key)[:2]:
    dictionary=i["data"]
    print(dictionary, end="\n\n")
    
    Info=relevantItemInfo(dictionary)
    print(Info, end="\n\n")

In [ ]:
dictList=[]

for i in zot.collections_sub(ID_Universities):
    name,key=i["data"]["name"],i["data"]["key"]
    print(f"name: {name} --> key: {key}")
    
    for i in zot.collection_items(key):
        # only use data if there is an attachment.
        if i["data"]["itemType"]=='attachment':
            Info=relevantItemInfo(i["data"])
            dictList+=[{"Uni": name,"FileKey":Info["key"],"url":Info["url"]}]
    


In [ ]:
df=pd.DataFrame(dictList)
df.head()

In [ ]:
def FileLinks(Filekey):
   #print(Filekey)
    import os
    DIR=f"C:\\Users\\mr\\Zotero\\storage\\{Filekey}"
    #print(DIR)
    FileNames=os.listdir(DIR)
    file=[i for i in FileNames if i.endswith(".html")]
    return DIR+"\\"+FileNames[-1]



df["file"]=df.FileKey.apply(FileLinks)
df.head()

In [ ]:
df.to_json("2023-07-11_Uni_Files_raw.json")

In [ ]:
df.columns

# Add Content from HTMLs

In [ ]:

#find_date("https://www.uu.nl/en/events/chat-gpt-discover-the-magic-of-ai")

def find_dateX(x):
    try:
        return find_date(x)
    except:
        print(x)
        print_time()

print_time()        
df["date"]=df.url.apply(find_dateX)
print_time()        

In [ ]:
df.to_json("2023-07-11_Uni_Files_raw_date.json")

In [2]:
df=pd.read_json("2023-07-11_Uni_Files_raw_date.json")

In [3]:
import MyLib.nlp as nlp
import MyLib.HTML_prep as HTML_prep
import MyLib.analysis as analysis 
%load_ext autoreload
%autoreload 2




In [4]:
def FileInfo(FilePath):
    Title,h1,h2,h3,text,linkName,linkUrl=None,None,None,None,None,None,None
    
    if FilePath.endswith(".html"):
    
        try:
            #print(FilePath)

            with open(FilePath,"r", encoding='utf-8') as f:
                text= f.read()
            
        except:
            print("error opening the html file. File does not exist?")
            return
        
        
        try:
            from bs4 import BeautifulSoup

            soup = BeautifulSoup(text, "html5lib") #'html.parser')       
            Title=soup.title.text
            
            
            #soup=HTML_prep.removeIMGs(soup,IMG_filename=Title.replace(" ","_"))      
            All_divs=[p.parent for p in soup("p") if p.parent.name=="div"]
            
            # All_divs=(list(set(All_divs)))
            
            All_divs=list(dict.fromkeys(All_divs))
            
            
            # Remove images
            try:
            
                IMG_filename="".join(x for x in Title.strip().replace(" ","_") if x.isalnum() or x=="_")[:100]
                All_divs=[HTML_prep.removeIMGs(div,write_img=False,IMG_filename=IMG_filename) for div in All_divs]        

            except:
                print(f"error removing images: {FilePath}")
                
            h1=[item for sublist in [[i.text for i in div("h1")] for div in All_divs if div("h1")!=None] for item in sublist]
            h2=[item for sublist in [[i.text for i in div("h2")] for div in All_divs if div("h2")!=None] for item in sublist]
            h3=[item for sublist in [[i.text for i in div("h3")] for div in All_divs if div("h3")!=None] for item in sublist]
        

            # here only get all divs
            #text=[item for sublist in [[i.text for i in div("p")] for div in All_divs if div("p")!=None] for item in sublist]
            
            # here gets all human readable text parts --> includes headlines. the \n \n keeps distance to headlines
            text=[i.get_text().replace("\n ",". ") for i in All_divs if i.get_text()!=None] # --> I use the \n \n later to split sentences
            
            linkName=[item for sublist in [[i.text for i in div("a")] for div in All_divs if div("a")!=None] for item in sublist]
            linkUrl=[item for sublist in [[i.get('href') for i in div("a")] for div in All_divs if div("a")!=None] for item in sublist]
            
        except:
            print(f"error with: {FilePath}", end=". ")
            
            try:
                text=text.split("\n\n")
 
                print("--> text from reading as a text file.")
            except:
                print("also no text file")
                
        #leave out words longer than 100 characters to avoid undetected embedded images and other shit.
        text=[" ".join([y.replace("\n"," ").strip() for y in i.split(" ") if len(y)<100]) for i in text] 
       # text=["".join([y for y in i if len(y)<100]) for i in text] 
        
    return pd.Series([Title,h1,h2,h3,text,linkName,linkUrl])
        
        
#df.file[2:6].apply(FileInfo)
print_time()        
df[["Title","h1","h2","h3","text","linkName","linkUrl"]]=df.file.apply(FileInfo)
print_time()        

Current Time = 15:25:46
error with: C:\Users\mr\Zotero\storage\RD8VNVZD\nrc-digitaledieven-23-02-08.html. --> text from reading as a text file.
error removing images: C:\Users\mr\Zotero\storage\MDTRC7HH\AI In Education.html
Current Time = 15:27:51


In [ ]:
len(df.text.dropna())

In [5]:
df.to_json("2023-07-11_Uni_Files_NLP.json")

In [181]:
df=pd.read_json("2023-07-11_Uni_Files_NLP.json")



In [182]:
def Insert_title(text,Title):
    if Title!=None and type(text)==list:
        text=[Title.strip()]+text
        return text
    else:
        return text


df["text"]=df[["text","Title"]].apply(lambda x: Insert_title(*x),axis=1)

In [190]:
def Splitter(paragraphs, max_len=5000):
    
    sentences=[]
    paragraphs_out=[]
    if isinstance(paragraphs,list):
        
        for paragraph in paragraphs:
            paragraph_out=[] 
       
            from nltk.tokenize import sent_tokenize
            paragraph=paragraph.replace("..",". ")
            
            sentence=sent_tokenize(paragraph) # per paragraph
            sentence=[s for s in sentence if len(s)>1]

            counter = 0
            P="" # will be filled with sentences untill it has 5000 characters.
            for s in sentence:
                counter += len(s) # calculate the lenght if it would be added.

                if counter > max_len: # does not add the sentence to the paragraph if too long
                    
                    paragraph_out.append(P)
                    
                    L=len(paragraph_out[-1])
                    print(f"{L} + {len(s)} = {counter} --> split {len(paragraph_out)} times.")
                
                    counter=0
                    P=""
                    
                if counter < max_len:
                    P=P.strip()+" "+ s.strip()
            
            paragraph_out.append(P)
            
            sentences+=sentence
            
            paragraphs_out.extend(paragraph_out)
                
                
    sentences_len=[len(i) for i in sentences]
    paragraphs_len=[len(i) for i in paragraphs_out]
 
        
    return pd.Series([paragraphs_out,sentences,paragraphs_len,sentences_len])


df[["paragraphs","sentences","paragraphs_len","sentences_len"]]=df["text"].apply(Splitter, max_len=4500)
df["paragraph_sum"]=df.paragraphs_len.apply(lambda x: sum(x))

df["text"][:4].apply(Splitter)

4470 + 115 = 4542 --> split 1 times.
4509 + 140 = 4618 --> split 1 times.
4542 + 141 = 4638 --> split 1 times.
4543 + 47 = 4546 --> split 1 times.
4472 + 216 = 4645 --> split 1 times.
4383 + 236 = 4599 --> split 1 times.
4462 + 90 = 4534 --> split 1 times.
4547 + 55 = 4551 --> split 1 times.
4462 + 100 = 4531 --> split 1 times.
4349 + 199 = 4518 --> split 1 times.
4438 + 189 = 4596 --> split 1 times.
4518 + 212 = 4687 --> split 1 times.
4498 + 121 = 4577 --> split 1 times.
4372 + 281 = 4617 --> split 1 times.
4695 + 250 = 4639 --> split 2 times.
4706 + 171 = 4598 --> split 3 times.
4534 + 364 = 4694 --> split 4 times.
4449 + 315 = 4730 --> split 1 times.
4341 + 216 = 4518 --> split 1 times.
4529 + 94 = 4582 --> split 1 times.
4480 + 60 = 4502 --> split 1 times.
4392 + 274 = 4645 --> split 1 times.
4711 + 243 = 4654 --> split 2 times.
4749 + 225 = 4697 --> split 3 times.
4637 + 163 = 4545 --> split 4 times.
4550 + 175 = 4538 --> split 5 times.
4620 + 182 = 4608 --> split 6 times.
4502 +

,0,1,2,3
0,[],[],[],[]
1,"[ Research projects - Communication Science, T...","[Research projects - Communication Science, Th...","[42, 135, 1403, 120]","[41, 62, 72, 16, 22, 70, 19, 57, 43, 31, 34, 1..."
2,[ INtoEAT: Chatbot algorithms and coaching str...,[INtoEAT: Chatbot algorithms and coaching stra...,"[95, 1567, 114]","[94, 86, 88, 73, 158, 225, 116, 115, 143, 104,..."
3,[],[],[],[]


In [184]:
# Check the lenght of paragraphs

df[df.paragraphs_len.apply(lambda x: any(i > 5000 for i in x))]

,Uni,FileKey,url,file,date,Title,h1,h2,h3,text,linkName,linkUrl,paragraphs,sentences,paragraphs_len,sentences_len


In [199]:
# just checkin if the long paragraphs are necessary...
df[df["paragraph_sum"]>50000]

df=df[df.Title!='Research Posters - Faculty of Geosciences - Utrecht University']


In [200]:
df.to_json("2023-07-12_Uni_Files_NLP_splitter.json")

# Explode --> by sentence or by paragraph?

In [203]:
#df=df.explode("sentences").drop_duplicates(subset="sentences").reset_index(drop=True)
df.paragraphs[1]

[' Research projects - Communication Science',
 'This website uses functional and non-privacy invading cookies. Do you also accept the placement of other cookie types?More information.',
 'Staff projects:. Information Explosion. INtoEAT: Chatbot algorithms and coaching strategies for FoodCoach Liz. Media and Morality. MyMovez and SocialMovez: Social Network Health Campaigns. New\xa0reality and human-computer interaction. Social Human Agent Interaction. Tailored Persuasive Communication. PhD projects. Look Who’s Talking: Towards Engaging Long-Term Interactions with Conversational Agents(Erkan Basar). Understanding and reducing conformity to cyber aggression among adolescents in mobile messaging apps (Daniëlle Bleize). An Understanding Virtual Coach for People with Amnesia\xa0(Roel Boumans). Investigating Coping Strategies and News Avoidance: Developing an Intervention to Empower Children in Coping with Negative News(Ming Ebbinkhuijsen). Chatbot Usability - Studying the Impact of Chatbot

### I will filter in the analysis file. Therefore this code is not needed anymore...

ChatGPT_Terms="ChatGPT, Chat-GPT, GPT3, GPT-3, GPT-x, GPT-4, GPT4,\
Transformer, OpenAI, AI, hallucination, Text generation, LLM, GPT, Chatbot, Models, generative, Intelligence, Model"

import MyLib.nlp as nlp

df["AI_paragraphs"]=df.paragraphs.dropna().apply(nlp.filter_paragraphs,by=ChatGPT_Terms).dropna()

df["AI_Paragraphs_len"]=df["AI_paragraphs"].apply(lambda x: [len(i) for i in x])
df[df.AI_Paragraphs_len.apply(lambda x: any(i > 4999 for i in x))]


In [207]:
column="sentences"

df=df[df.file.apply(lambda x: x.endswith("html"))]


df=df.explode(column).reset_index(drop=True)
df=df[df[column].apply(lambda x: type(x)==str)]
df=df.drop_duplicates(column)
df["LEN"]=df[column].apply(lambda x: len(x.split(" ")))


In [218]:

df=nlp.NLP_Pipeline(df, sentiment=False, language="en",translate=True, column=column)

Current Time = 16:19:11
Lenght:  10607
cleaning done.
Current Time = 16:19:11
Current Time = 16:20:07
language detection done.
Translating...
nl Research c
nl Research t
nl Dit projec
nl Communicat
nl Van virtua
nl Dat geeft 
nl Communicat
nl Tot vorige
nl Er was een
nl Het intern
nl Sociale me
nl Waar dat a
nl En daar zi
nl ‘Sinds de 
nl ‘In prakti
nl Het maakt 
nl ‘Boodschap
nl Hoe zit he
nl En kan wat
nl Begrijp Ke
nl Hij licht 
nl Het ervare
nl ‘Deze tech
nl Ondertusse
nl Momenteel 
nl ‘Een risic
nl Sociale VR
nl Over de mo
nl Gaan intel
nl ‘Deels wel
nl AI is nog 
nl ’Ketelaar 
nl De app reg
nl Hoe maak j
nl Door spele
nl ‘Een trap 
nl Gamificati
nl Mensen doe
nl ‘Een aantr
nl Of een cha
nl ‘Stel je k
nl Dan is de 
nl Maar wat a
nl Is dat nog
nl De ene tec
nl Gamificati
nl Toch zijn 
nl We moeten 
nl Er zullen 
nl Communicat
nl Vanaf 15 n
nl Interessan
nl Radboud Un
nl . . Gerela
nl . . Bekijk
nl ChatGPT, b
nl Plotseling
nl Het is nam
nl ChatGPT ge
nl Hele tekst
nl Dit kan ti
nl D

nl ‘Op master
nl ‘Ik wil st
nl Tiberius: 
nl Ik ben dan
nl Kan AI een
nl De gevaren
nl We staan v
nl We vinden 
nl Van Duijn:
nl Kouwenhove
nl Dat is ris
nl Van Duijn 
nl Van Duijn:
nl Ik denk da
nl Het aankom
nl Chatten me
nl . . . . Gr
nl Voorbeelde
nl Daarnaast 
nl De chatbot
nl Ze werd 10
nl Ze leefde 
nl Er kunnen 
nl In het ond
nl Leerlingen
nl Doordat de
nl Chatbots d
nl Nadira Saa
nl Daarnaast 
nl . . voortg
nl AI in het 
nl Wat kunnen
nl ‘We bevind
nl De vooruit
nl Kijk maar 
nl ‘Het is be
nl Maar uitei
nl Meer weten
nl Workshop C
nl Op donderd
nl Lieve de W
nl De komst v
nl Heeft het 
nl In deze wo
nl Ze laat ee
nl Concrete v
nl Er is vold
nl Als je geï
nl Let op: de
nl Send an em
nl Notarieel 
nl Notarieel 
nl Rechtsgele
nl Leidse stu
nl Het viel h
nl „Ons polit
nl Je krijgt 
nl Lees de ve
nl . . verkie
nl In hoeverr
nl In een rec
nl Het gehype
nl Toch blijf
nl . . bestuu
nl . . interd
nl 24 februar
nl Hoe gaan w
nl Binnen het
nl Wanneer zi
nl Het raakt 
nl Naast het 
nl Hie

nl Het maken 
nl Antwoorden
nl Ze vroeg C
nl En die ant
nl Dat klinkt
nl Er zijn vo
nl “Teksten g
nl Verder wor
nl “Sterker n
nl En als je 
nl Gespreksle
nl Waarheidsv
nl Terwijl ee
nl OpenAI, he
nl Gijsbers l
nl De chatbot
nl Ook doet h
nl “Als je Ch
nl De chatbot
nl Bronnen wo
nl Een nieuw 
nl “Hoewel de
nl Worden zij
nl “Ja, voor 
nl Gewoon wee
nl “Het kost 
nl Binnen afz
nl Die dachte
nl Volgens fi
nl “Ik denk d
nl Geesteswet
nl De afgelop
nl Unieke tek
nl Het is een
nl AI roept n
nl Kan AI hel
nl Maar ook: 
nl Hiernaar d
nl Met een ac
nl Bijvoorbee
nl Er valt ve
nl In Radio1-
nl En hij ste
nl Het punt i
nl “Ook de be
nl Ze kunnen 
nl Vooringeno
nl In NRC bes
nl Maar hij z
nl Een proble
nl Hierover s
nl Hier is bi
nl “Een compl
nl Examencomm
nl !’, De Jor
nl "Focus eve
nl “Als schri
nl “Duidelijk
nl Of en hoe 
nl De open be
nl (NRC, 28 a
nl . . extern
nl       Chat
nl Universita
nl In een opi
nl “Ik ben va
nl Datagedrev
nl Ik zou ied
nl Geef de ch
nl Voed hem m
nl "Over de o
nl Het

nl De discuss
nl En er mag 
nl Of als des
nl “Ik heb ve
nl Wat beteke
nl Vaak wordt
nl Bij het be
nl Juist omda
nl De wetensc
nl Omdat we n
nl Voor die t
nl “Alleen al
nl “Als we ee
nl “We gaan A
nl Lin droomt
nl En het voo
nl Gaat AI de
nl Zet hun in
nl Er zit een
nl “Het denke
nl We weten n
nl Het leven 
nl Voor de ee
nl Zo zal het
nl Chatbot in
nl . . Waar d
nl Onder ande
nl De voor- e
nl Op de afde
nl Chatbots z
nl Dat verloo
nl Een algori
nl Een assist
nl Laten we e
nl Een sector
nl Niet metee
nl En het te 
nl Een van de
nl En dan nie
nl Vraagbaak 
nl Een voorbe
nl Flow.ai on
nl Via Whatsa
nl De chatbot
nl Met die tr
nl Lukte het 
nl “Door het 
nl Digitale c
nl De huisart
nl Mensen met
nl Maar om ee
nl Bovendien 
nl Een chatbo
nl “We zien a
nl Patiënten 
nl “De arts b
nl Als dat ni
nl Een chatbo
nl Al zijn de
nl “Als het a
nl Snel advie
nl “Het syste
nl Lambèr Roy
nl Al moeten 
nl Noodnummer
nl “Als de sp
nl Een algori
nl Zo kan hij
nl Robotethic
nl Door razen
nl Dat redt l
nl Lu 

nl Voorheen s
nl En als iem
nl Dat algori
nl Tegenwoord
nl Daarmee ku
nl . . . . . 
nl Hoeft dat 
nl . . . . . 
nl In de soft
nl Maar niet 
nl Soms is er
nl Geert-Jan 
nl In softwar
nl Daar zit d
nl Neem een z
nl Een gedeel
nl Maar er is
nl Dan kan je
nl Dus daar z
nl . . . . . 
nl . . . . . 
nl Je kunt ze
nl Sinds nove
nl Daardoor i
nl Geert-Jan 
nl In plaats 
nl Die is ook
nl Op beide a
nl Wat ze pre
nl . . . . . 
nl Zij voldoe
nl Dus alle t
nl In het sch
nl En dat kun
nl Daar doen 
nl We kijken 
nl Denk aan A
nl Wel is er 
nl Antwoorden
nl De toekoms
nl . . . . . 
nl Eerst zoch
nl Toen ontde
nl Bijvoorbee
nl Dat lijkt 
nl We zullen 
nl . . . . . 
nl Welke onde
nl . . . . . 
nl We kijken 
nl Hoe kunnen
nl Software v
nl Als een on
nl Wat we met
nl Dat model 
nl Uit eerder
nl Daardoor v
nl Naast het 
nl Daarbij pu
nl Geert-Jan 
nl Waarmee we
nl De moderne
nl Het levert
nl Bij de TU 
nl Dat gaat e
nl Dat onderz
nl Dus hoewel
nl . . . . . 
nl . . . . . 
nl Wij moeten
nl In het pro
nl Als

In [219]:
df.to_json("2023-07-12_ChatGPT_Sentences_NLP-Out.json")

In [ ]:
df.columns

df2=df[['Uni','FileKey','url','date', 'Title','linkName', 'linkUrl', 'AI_paragraphs','text_clean', 'letters_count', 'word_count',
       'language', 'source_language', 'pure_text', 'Lemmata', 'NoStopwords']]

In [ ]:
df2.to_json("2023-06-06_Zotero_AI_nlp_en2.json")